<a href="https://colab.research.google.com/github/coding-cosmos/AB-Testing/blob/main/AB_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Loading Data


In [ ]:
import pandas as pd
import datetime
from datetime import date, timedelta
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
pio.templates.default = 'plotly_white'

control_data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/AB Testing/control_group.csv",sep = ';')
test_data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/AB Testing/test_group.csv",sep = ';')

In [ ]:
control_data.head()

,Campaign Name,Date,Spend [USD],# of Impressions,Reach,# of Website Clicks,# of Searches,# of View Content,# of Add to Cart,# of Purchase
0,Control Campaign,1.08.2019,2280,82702.0,56930.0,7016.0,2290.0,2159.0,1819.0,618.0
1,Control Campaign,2.08.2019,1757,121040.0,102513.0,8110.0,2033.0,1841.0,1219.0,511.0
2,Control Campaign,3.08.2019,2343,131711.0,110862.0,6508.0,1737.0,1549.0,1134.0,372.0
3,Control Campaign,4.08.2019,1940,72878.0,61235.0,3065.0,1042.0,982.0,1183.0,340.0
4,Control Campaign,5.08.2019,1835,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
test_data.head()


,Campaign Name,Date,Spend [USD],# of Impressions,Reach,# of Website Clicks,# of Searches,# of View Content,# of Add to Cart,# of Purchase
0,Test Campaign,1.08.2019,3008,39550,35820,3038,1946,1069,894,255
1,Test Campaign,2.08.2019,2542,100719,91236,4657,2359,1548,879,677
2,Test Campaign,3.08.2019,2365,70263,45198,7885,2572,2367,1268,578
3,Test Campaign,4.08.2019,2710,78451,25937,4216,2216,1437,566,340
4,Test Campaign,5.08.2019,2297,114295,95138,5863,2106,858,956,768


#Data Preparation

### Fix columns names

In [ ]:
control_data.columns =["Campaign Name", "Date", "Amount Spent",
                        "Number of Impressions", "Reach", "Website Clicks",
                        "Searches Received", "Content Viewed", "Added to Cart",
                        "Purchases"]


test_data.columns = ["Campaign Name", "Date", "Amount Spent",
                        "Number of Impressions", "Reach", "Website Clicks",
                        "Searches Received", "Content Viewed", "Added to Cart",
                        "Purchases"]

### Check null values

In [ ]:
control_data.isnull().sum()

,0
Campaign Name,0
Date,0
Amount Spent,0
Number of Impressions,1
Reach,1
Website Clicks,1
Searches Received,1
Content Viewed,1
Added to Cart,1
Purchases,1


In [ ]:
test_data.isnull().sum()

,0
Campaign Name,0
Date,0
Amount Spent,0
Number of Impressions,0
Reach,0
Website Clicks,0
Searches Received,0
Content Viewed,0
Added to Cart,0
Purchases,0


### Fix the null values in control_data

- Using mean to fill the null values

In [ ]:
control_data["Number of Impressions"]=control_data["Number of Impressions"].fillna(value = control_data["Number of Impressions"].mean())
control_data["Reach"] = control_data["Reach"].fillna(value = control_data["Reach"].mean())

control_data["Website Clicks"] = control_data["Website Clicks"].fillna(value = control_data["Website Clicks"].mean())
control_data["Searches Received"] = control_data["Searches Received"].fillna(value = control_data["Searches Received"].mean())

control_data["Content Viewed"] = control_data["Content Viewed"].fillna(value=control_data["Content Viewed"].mean())
control_data["Added to Cart"] = control_data["Added to Cart"].fillna(value=control_data["Added to Cart"].mean())
control_data["Purchases"] = control_data["Purchases"].fillna(value=control_data["Purchases"].mean())

- Sanity check

In [ ]:
control_data.isnull().sum()

,0
Campaign Name,0
Date,0
Amount Spent,0
Number of Impressions,0
Reach,0
Website Clicks,0
Searches Received,0
Content Viewed,0
Added to Cart,0
Purchases,0


### Merge the both dataset

In [ ]:
ab_data = control_data.merge(test_data,how='outer').sort_values(["Date"])
ab_data = ab_data.reset_index(drop = True)

ab_data

/tmp/ipython-input-1626133578.py:1: UserWarning:

You are merging on int and float columns where the float values are not equal to their int representation.



,Campaign Name,Date,Amount Spent,Number of Impressions,Reach,Website Clicks,Searches Received,Content Viewed,Added to Cart,Purchases
0,Control Campaign,1.08.2019,2280,82702.000000,56930.000000,7016.000000,2290.000000,2159.000000,1819.0,618.000000
1,Test Campaign,1.08.2019,3008,39550.000000,35820.000000,3038.000000,1946.000000,1069.000000,894.0,255.000000
2,Control Campaign,10.08.2019,2149,117624.000000,91257.000000,2277.000000,2475.000000,1984.000000,1629.0,734.000000
3,Test Campaign,10.08.2019,2790,95054.000000,79632.000000,8125.000000,2312.000000,1804.000000,424.0,275.000000
4,Control Campaign,11.08.2019,2490,115247.000000,95843.000000,8137.000000,2941.000000,2486.000000,1887.0,475.000000
5,Test Campaign,11.08.2019,2420,83633.000000,71286.000000,3750.000000,2893.000000,2617.000000,1075.0,668.000000
6,Control Campaign,12.08.2019,2319,116639.000000,100189.000000,2993.000000,1397.000000,1147.000000,1439.0,794.000000
7,Test Campaign,12.08.2019,2831,124591.000000,10598.000000,8264.000000,2081.000000,1992.000000,1382.0,709.000000
8,Control Campaign,13.08.2019,2697,82847.000000,68214.000000,6554.000000,2390.000000,1975.000000,1794.0,766.000000
9,Test Campaign,13.08.2019,1972,65827.000000,49531.000000,7568.000000,2213.000000,2058.000000,1391.0,812.000000
